In [1]:
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
import pandas as pd
import json
from pandas.io.json import json_normalize
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
X = [[0., 0.], [1., 1.]]

In [4]:
y = [0, 1]
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)

clf.fit(X, y)  

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [5]:
clf.predict([[-5, 1], [-13., -2.]])

array([0, 0])

In [6]:
[coef.shape for coef in clf.coefs_]

[(2, 5), (5, 2), (2, 1)]

In [2]:
dfStockData = pd.read_json("data/TSLA_stocks.json").transpose()
dfStockData.head()

,1. open,2. high,3. low,4. close,5. volume
2018-11-05,340.50,343.9550,330.140,341.40,7829954.0
2018-11-02,343.74,349.2000,340.910,346.41,7807971.0
2018-11-01,338.26,347.8400,334.725,344.28,8000132.0
2018-10-31,332.54,342.0000,329.100,337.32,7624348.0
2018-10-30,328.39,337.8999,322.260,329.90,9126704.0


In [49]:
for i in range(11275):
    df = json.load(open("data/messages/resp_%s.json" % i))
    dftemp = json_normalize(df['messages'])
    dfTweet = dfTweet.append(dftemp[['body', 'created_at']], ignore_index=True)
print(dfTweet)

                                                     body  \
0       $TSLA very small put add today. Waiting for th...   
1                   $TSLA target 370 for the short Again?   
2               Whole new company now shorts beware $TSLA   
3              $TSLA Scary scary 🐻 TSLA’s coming for you!   
4       $TSLA max pain is 315 for expiry 2018-11-02 So...   
5       $TSLA Happy Halloween all. And this one is ded...   
6       $TSLA the action this week builds confidence. ...   
7       $TSLA AUTO PARK!! yes!!! When I gotta grab a q...   
8       $TSLA Charts show 344 is where this maxes out ...   
9                         $TSLA I love your kinda crazy 😜   
10      $TSLA jumps to #1 in global plug-in electric c...   
11      10-31-2018 Potential Whale Trades for $TSLA (L...   
12                       $TSLA chairmen announcement soon   
13      $TSLA there is no truth to the rumor I share t...   
14      $TSLA everyone want&#39;s the treat but will i...   
15      $TSLA will need 

In [88]:
vectorizer = CountVectorizer(stop_words='english')
vectorizer.fit_transform(dfTweet['body']).todense()
vectorizer.vocabulary_
print(max(vectorizer.vocabulary_))

years


In [58]:
dfTweet.to_csv("clean_tweet_data.csv")

In [59]:
df = pd.read_csv("clean_tweet_data.csv")
df.tail()

,Unnamed: 0,body,created_at
339464,339464,Here’s what 37 Estimize analysts believe $TSLA...,2018-03-11T23:28:45Z
339465,339465,$TSLA $SPY looking at him gets me high! Dude i...,2018-03-11T23:17:02Z
339466,339466,"$TSLA Insiders adding shares recntly, anyways ...",2018-03-11T23:11:54Z
339467,339467,"$TSLA not looking great. $321, $317 take it to...",2018-03-11T22:57:19Z
339468,339468,$TSLA been in this range for almost a year. RS...,2018-03-11T22:54:18Z


In [60]:
df.tail()

,Unnamed: 0,body,created_at
339464,339464,Here’s what 37 Estimize analysts believe $TSLA...,2018-03-11T23:28:45Z
339465,339465,$TSLA $SPY looking at him gets me high! Dude i...,2018-03-11T23:17:02Z
339466,339466,"$TSLA Insiders adding shares recntly, anyways ...",2018-03-11T23:11:54Z
339467,339467,"$TSLA not looking great. $321, $317 take it to...",2018-03-11T22:57:19Z
339468,339468,$TSLA been in this range for almost a year. RS...,2018-03-11T22:54:18Z


-----

In [5]:
dfStockData["diff"] = dfStockData["4. close"] - dfStockData["1. open"]

In [10]:
dfStockMin = dfStockData.drop(columns=["1. open", "2. high", "3. low", "4. close", "5. volume"])

In [20]:
dfStockMin.index.name = "date"
dfStockMin.reset_index(inplace=True)

In [34]:
dfStockMin["date"] = pd.to_datetime(dfStockMin["date"])

In [31]:
# Do min/max normalization on diff values, to get values from -1 to 1
scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
vals = dfStockMin["diff"].values
vals = scaler.fit_transform(vals.reshape(-1,1))
dfStockMin["diff"] = vals

In [35]:
dfStockMin.to_csv("normed_stockdata.csv")

---

In [1]:
# Using python libs cucco and unidecode as well as standard libs for text normalization
from cucco import Cucco
from html import unescape
import pandas as pd
import re
from unidecode import unidecode
cucco = Cucco()

In [2]:
dfTweet = pd.read_csv("clean_tweet_data.csv")

In [3]:
# Convert created_at to just dates without time
dfTweet["date"] = dfTweet["created_at"].str.split('T',expand=True)[0]
dfTweet["date"] = pd.to_datetime(dfTweet["date"])
dfTweet = dfTweet.drop(columns="created_at")

In [4]:
dfTweet = dfTweet.drop(columns="Unnamed: 0")

In [5]:
drop_words = ["tsla"]
remove_regex = re.compile(r"\b(?:{}|$\w+|\w\w?|\d+)\b".format("|".join(re.escape(w) for w in drop_words)), flags=re.IGNORECASE)

In [6]:
normalizations = [
    "replace_emails",
    "replace_urls",
    "remove_stop_words",
    "replace_emojis",
    "replace_symbols",
    "replace_hyphens",
    "replace_punctuation",
    "remove_extra_white_spaces"
]
def normalize_str(s):
    s = unidecode(unescape(s))
    # remove drop words, stock symbols, 1-2 letter words, and numbers
    s = remove_regex.sub("", s)
    s = cucco.normalize(s, normalizations).strip().lower()
    return s

In [7]:
dfTweet["body"] = dfTweet["body"].apply(normalize_str)

In [8]:
# Remove empty rows
dfTweet = dfTweet[dfTweet["body"].map(len) > 0]

In [9]:
print(dfTweet)

                                                     body       date
0             small put add today waiting clowns possibly 2018-10-31
1                                      target short again 2018-10-31
2                                   company shorts beware 2018-10-31
3                                  scary scary coming you 2018-10-31
4                                  max pain expiry source 2018-10-31
5       happy halloween all dedicated bearish friends ... 2018-10-31
6                    action week builds confidence friday 2018-10-31
7       auto park yes gotta grab quickie store car roa... 2018-10-31
8                 charts show maxes week surprised higher 2018-10-31
9                                        love kinda crazy 2018-10-31
10               jumps global plug electric car sales ytd 2018-10-31
11            potential whale trades largest trade volume 2018-10-31
12                                  chairmen announcement 2018-10-31
13      truth rumor share nuerolin

In [15]:
dfTweet.to_csv("normed_tweets.csv")

---

In [1]:
from collections import Counter
import numpy as np
import pandas as pd
import pickle
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor

In [2]:
dfTweet = pd.read_csv("normed_tweets.csv")
dfStock = pd.read_csv("normed_stockdata.csv")

In [3]:
# Get counts for words
counts = Counter()
for tweet in dfTweet["body"]:
    words = tweet.split(" ")
    for word in words:
        counts[word] += 1

# Now assign each word an index, and eliminate words with less than 50 occurrences
# as well as the most frequent words
most_frequent = [w for w,c in counts.most_common(9)]
indices = {}
i = 0
for k in counts:
    if counts[k] >= 50 and k not in most_frequent:
        indices[k] = i
        i += 1

In [4]:
# Now create word vectors, normalizing from 0-1 for each individually
# Each word vector will represent one day's worth of tweets.
# Also put the correct output data for the corresponding date
scaler = preprocessing.MinMaxScaler()
input_data = []
output_data = []
for date in set(dfTweet["date"]):
    stock = dfStock[dfStock["date"] == date]["diff"]
    if len(stock) > 0:
        output_data.append(stock.iloc[0])
        
        word_vec = [0.0] * len(indices)
        tweets = dfTweet[dfTweet["date"] == date]["body"]
        for tweet in tweets:
            words = tweet.split(" ")
            for word in words:
                if word in indices:
                    word_vec[indices[word]] += 1
        word_vec = scaler.fit_transform(np.reshape(word_vec, (-1,1)))
        input_data.append(word_vec.reshape(1,-1)[0])

# Now split the data so that we have 20% left to evaluate accuracy
split_point = int(len(output_data) * 0.8)
input_train = input_data[:split_point]
output_train = output_data[:split_point]
input_test = input_data[split_point:]
output_test = output_data[split_point:]
input_data = None
output_data = None

In [5]:
mlp = MLPRegressor(solver="adam", alpha=1e-5, hidden_layer_sizes=(len(indices)//2,int(np.sqrt(len(indices)))))
mlp.fit(input_train, output_train)

MLPRegressor(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(2274, 67), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [6]:
predictions = mlp.predict(input_test)
for a,b in zip(output_test, predictions):
    print("{:7.4f}".format(a),"{:7.4f}".format(b),"{:7.4f}".format(abs(b-a)))

 0.4098  0.2075  0.2023
-0.2120 -0.3241  0.1121
-0.4452 -0.2109  0.2343
-0.2678  0.3116  0.5794
 0.0820 -0.0143  0.0962
 0.0769 -0.2351  0.3120
-0.0306  0.0137  0.0442
-0.0801  0.0011  0.0812
-0.6841 -0.2402  0.4439
-0.7746 -0.4115  0.3631
-0.0807  0.2902  0.3709
-0.2838  0.0214  0.3052
-0.2164 -0.0580  0.1584
-0.3622 -0.0882  0.2739
-0.0815 -0.1789  0.0974
-0.4587  0.0396  0.4984
-0.1732 -0.0603  0.1129
 0.5900  0.0086  0.5814
-0.4875  0.1172  0.6048
-0.2700 -0.2511  0.0189
 0.0845 -0.0450  0.1294
-0.0773 -0.1317  0.0544
-0.0958  0.1610  0.2568
 0.1622 -0.1451  0.3073
-0.1804  0.0629  0.2433
 0.1653  0.0404  0.1249
-0.0788 -0.2256  0.1468
-0.3681 -0.1635  0.2047
-0.1895 -0.2280  0.0385
-0.5305  0.2215  0.7520
 0.0575 -0.1360  0.1935
-0.2848 -0.3315  0.0468
-0.2384 -0.2246  0.0137


In [7]:
np.sqrt(mean_squared_error(output_test, predictions))

0.3074249035801162

In [8]:
# Save the NN and training/test data to file
with open("indices.pkl", "wb") as f:
    pickle.dump(indices, f)
with open("orig_x_train.pkl", "wb") as f:
    pickle.dump(input_train, f)
with open("orig_y_train.pkl", "wb") as f:
    pickle.dump(output_train, f)
with open("orig_x_test.pkl", "wb") as f:
    pickle.dump(input_test, f)
with open("orig_y_test.pkl", "wb") as f:
    pickle.dump(output_test, f)
with open("original_nn.pkl", "wb") as f:
    pickle.dump(mlp, f)

---

In [ ]:
import pickle
from sklearn.neural_network import MLPRegressor

In [2]:
# Open the indices for word vectors
with open("indices.pkl", "rb") as f:
    indices = pickle.load(f)

In [3]:
# Open the training data
with open("orig_x_train.pkl", "rb") as f:
    input_train = pickle.load(f)
with open("orig_y_train.pkl", "rb") as f:
    output_train = pickle.load(f)

In [4]:
# Open the test data
with open("orig_x_test.pkl", "rb") as f:
    input_test = pickle.load(f)
with open("orig_y_test.pkl", "rb") as f:
    output_test = pickle.load(f)

In [25]:
# Open the NN
with open("original_nn.pkl", "rb") as f:
    mlp = pickle.load(f)